Обычно, мы сохраняем обученную модель для дальнейшего использования.

Хорошо сохранять не только модель, но и некоторые методы, которые используются с ней (скалирование и т.п.)

Результаты методов normalization или standardization требуют тех же параметров от входящих данных (мин, макс, стд, сред), вы должны проверять это тестами или сохранять вместе с моделью.



In [1]:
from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split

# рандомный дата сет и его разделение на обуч/тест
X, y = make_blobs(n_samples=100, centers=2, n_features=2, random_state=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)

# сравним обуч / тест
for i in range(X_test.shape[1]):
    print('>%d, значения обуч.: min=%.3f, max=%.3f, test: min=%.3f, max=%.3f' %
         (i, X_train[:, i].min(), X_train[:, i].max(),
             X_test[:, i].min(), X_test[:, i].max()))

>0, значения обуч.: min=-11.856, max=0.526, test: min=-11.270, max=0.085
>1, значения обуч.: min=-6.388, max=6.507, test: min=-5.581, max=5.926


# Scale

In [2]:
from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# повторяем
X, y = make_blobs(n_samples=100, centers=2, n_features=2, random_state=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)

# МинМах - скалирование каждого значения от 0 до 1
scaler = MinMaxScaler()
scaler.fit(X_train)

# применяем к наборам данных
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

for i in range(X_test.shape[1]):
    print('>%d, значения обуч.: min=%.3f, max=%.3f, test: min=%.3f, max=%.3f' %
         (i, X_train_scaled[:, i].min(), X_train_scaled[:, i].max(),
             X_test_scaled[:, i].min(), X_test_scaled[:, i].max()))

>0, значения обуч.: min=0.000, max=1.000, test: min=0.047, max=0.964
>1, значения обуч.: min=0.000, max=1.000, test: min=0.063, max=0.955


# Применяем к модели

In [3]:
from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from pickle import dump

X, y = make_blobs(n_samples=100, centers=2, n_features=2, random_state=1)
X_train, _, y_train, _ = train_test_split(X, y, test_size=0.33, random_state=1)

scaler = MinMaxScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)

# определяем модель 
model = LogisticRegression(solver='lbfgs')
model.fit(X_train_scaled, y_train)

# сохраняем модель
dump(model, open('model.pkl', 'wb'))
# сохраняем скалирование 
dump(scaler, open('scaler.pkl', 'wb'))

# Проверяем

In [4]:
from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from pickle import load

X, y = make_blobs(n_samples=100, centers=2, n_features=2, random_state=1)
_, X_test, _, y_test = train_test_split(X, y, test_size=0.33, random_state=1)

# загружаем модель
model = load(open('model.pkl', 'rb'))
scaler = load(open('scaler.pkl', 'rb'))

print('Тест на необработанных данных')
for i in range(X_test.shape[1]):
    print('>%d, min=%.3f, max=%.3f' % (i, X_test[:, i].min(), X_test[:, i].max()))

# без скалирования
yhat = model.predict(X_test)
acc = accuracy_score(y_test, yhat)
print('Test Accuracy:', acc)    

print('\n')
# скалируем данные 
X_test_scaled = scaler.transform(X_test)
print('Тест на скалируемых данных')
for i in range(X_test_scaled.shape[1]):
    print('>%d, min=%.3f, max=%.3f' % (i, X_test_scaled[:, i].min(), X_test_scaled[:, i].max()))

yhat = model.predict(X_test_scaled)
acc = accuracy_score(y_test, yhat)
print('Test Accuracy:', acc)

Тест на необработанных данных
>0, min=-11.270, max=0.085
>1, min=-5.581, max=5.926
Test Accuracy: 0.9696969696969697


Тест на скалируемых данных
>0, min=0.047, max=0.964
>1, min=0.063, max=0.955
Test Accuracy: 1.0
